In [1]:
%pip install scikit-learn


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short
import numpy as np
import sys
sys.path.append('../')
from utils import remove_similar_rows_per_player, find_lines_with_player, del_patterns, map_emoji_to_description, remove_accents, name_wordgroups
import emoji
from googletrans import Translator


In [3]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

# First Transformation (with Pattern deletion)
The first transformation focus on the deletion of Patterns to clean the corpus.

## Filter out German data

In [4]:
# Filter out the German data and reindex
df_ger = df[df["language"] == "de"]
df_ger = df_ger.reset_index(drop=True)

In [5]:
# Remove the similiar rows (The Function is imported from utils on top)
df_before = remove_similar_rows_per_player(df_ger, df_ger['player'].unique())

In [6]:
# transform Hincapié to Hincapie
df_before.loc[df_before['player'] == 'Piero Hincapié', 'player'] = 'Piero Hincapie'

## Transform Data to lower case

In [7]:
# transform data into lower case
data_lower = df_before.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()



## Delte Patterns

In [8]:
# delete content patterns
data_wo_pattern = data_lower.copy()
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))

In [9]:
pattern = ['nutze kicker', 
           'mit dem', 
           'nur €2,49', 
           'bereits pur-abonnent?', 
           'alle antworten', 
           'hinweis zur verarbeitung', 
           'werbung','olympia-verlags', 
           'bild', 'g+j medien gmbh', 
           'services',
           'kurz-link dieses artikels lautet:', 
           'http://epaper.welt.de',
           'stephan von nocks',
           'www.faz.net',
           'mcfit mitgliedschaft',
           'fitx-vertrag',
           'kündigeneasyfitness',
           'proteinbedarf',
           'fitnessland',
           'kündigeneasyfitness',
           'trainingspuls berechnen',
           'pulsrechner',
           'fitseveneleven-mitgliedschaft',
           'alkoholabbau & promille',
           'index.promillerechner',
           'mitgliedschaft kündigen',
           'promillerechner',
           'ihr body-mass-index',
           'bmi rechner',
           'kalorienrechner',
           'grundumsatz & kalorienbedarf',
           'partnerangebote',
           'newsletter',
           'journalismus der presse',
           'abonnieren',
           '(apa)',
           'foto:',
           'quelle:',
           'lesezeit:',
           'lesen sie mehr',
           'stellenmarkt der sz.',
           'bewerben sie sich jetzt',
           'gutscheine',
           'vergleichsportal',
           'stern plus-inhalten',
           'jederzeit kündbar',
           'bereits registriert?',
           'zur startseite',
           'öffnet in neuem tab oder fenster',
           'vollansicht der tabelle unter'        
           ]

In [10]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: del_patterns(str(x), pattern))

## Transform Emojis to text

In [11]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'de',), str(x)))

## Remove noise

In [12]:
data_rm_1 = data_wo_pattern.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm_1['data'] = data_rm_1['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm_1['data'] = data_rm_1['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm_1['data'] = data_rm_1['data'].apply(strip_multiple_whitespaces)

#strip spanish accents
data_rm_1['data'] = data_rm_1['data'].apply(lambda x: remove_accents(str(x)))

#strip_short deletes words smaller 3
data_rm_1['data'] = data_rm_1['data'].apply(strip_short)
 
#strip links
data_rm_1['data'] = data_rm_1['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# Remove stopwords

In [13]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts']

stop_words_to_add = ['vor', 'in', 'den', 'dem', 'beim', 'wir', 'der', 'ist','ende', 'seite', 'ersten', 'fürs', 'eh', 'blick', 'schon', 'zumal', 'erst', 'mal', 'bild', 't', '(dpa)']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

for word in stop_words_to_add:
  german_stop_words.append(word)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm_1['data'] = data_rm_1['data'].apply(lambda x: remove_stopwords(x, german_stop_words))


In [14]:
data_rm_1['data'].iloc[0]

'trainer xabi alonso mitgereisten fans monaco familie genoss xabi alonso team hotel monaco moment erfolges dramatischen einzug achtelfinale europa league ,,aber gab zeit feiern" sagte trainer bayer leverkusen personlich wohl wichtigste spiel jungen trainerlaufbahn erlebt alonso ,,es intensiv fur situation ruhig fokussiert besten entscheidungen treffen bessere emotionale kontrolle besser spiel leider konnten nicht verteidigen zufrieden leistung mannschaft gross anspannung elfmeterschiessen alonso ,,ehrlich vertrauen mittwoch training elfmeter geubt gut azmoun gut amiri gut klare idee spieler schiessen konnte bundesliga viele elfer verschossen funfmal folge getroffen effektiv passieren spieler gut gemacht gibt trick lotto gestern glucklichen glucksmoment exequiel palacios hebt tor euro highlight geht bundesliga sonntag uhr freiburg ,,wir kennen freiburg gerade guten moment spielen hohen intensitat ahnlich mainz vielleicht spiel mainz gute schule fur fur freiburg spiel" sagt alonso bayer 

In [15]:
data_rm_1.reset_index(drop=True, inplace=True)


In [16]:
data_rm_1['data'].iloc[0]

'trainer xabi alonso mitgereisten fans monaco familie genoss xabi alonso team hotel monaco moment erfolges dramatischen einzug achtelfinale europa league ,,aber gab zeit feiern" sagte trainer bayer leverkusen personlich wohl wichtigste spiel jungen trainerlaufbahn erlebt alonso ,,es intensiv fur situation ruhig fokussiert besten entscheidungen treffen bessere emotionale kontrolle besser spiel leider konnten nicht verteidigen zufrieden leistung mannschaft gross anspannung elfmeterschiessen alonso ,,ehrlich vertrauen mittwoch training elfmeter geubt gut azmoun gut amiri gut klare idee spieler schiessen konnte bundesliga viele elfer verschossen funfmal folge getroffen effektiv passieren spieler gut gemacht gibt trick lotto gestern glucklichen glucksmoment exequiel palacios hebt tor euro highlight geht bundesliga sonntag uhr freiburg ,,wir kennen freiburg gerade guten moment spielen hohen intensitat ahnlich mainz vielleicht spiel mainz gute schule fur fur freiburg spiel" sagt alonso bayer 

## Store csv

In [17]:
data_rm_1 = name_wordgroups(data_rm_1)

In [18]:
data_rm_1.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_1.csv', index=False)

# Second Transformation (setence with Playernames)
The second transformation focus on the deletion of sentences to clean the corpus.

## Get lines and following lines where the Player name appears in the corpus 

In [19]:
# transform data into lower case
data_lower = df_before.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()

In [20]:
# delete content pattern and commercials
data_lower['data'] = data_lower['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_lower['data'] = data_lower['data'].apply(lambda x: del_patterns(str(x), pattern))

In [21]:
# select only paragraphs which include playernames 
data_with_playernames = find_lines_with_player(data_lower, data_lower['player'].unique(), n_lines = 1)

## Transform Emojis

In [22]:
# Transform emojis into german text
data_with_playernames['data'] = data_with_playernames['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'de',), str(x)))

## Delete unnecessary

In [23]:
data_rm_2 = data_with_playernames.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm_2['data'] = data_rm_2['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm_2['data'] = data_rm_2['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm_2['data'] = data_rm_2['data'].apply(strip_multiple_whitespaces)

#strip spanish accents
data_rm_2['data'] = data_rm_2['data'].apply(lambda x: remove_accents(str(x)))

#strip_short deletes words smaller 3
data_rm_2['data'] = data_rm_2['data'].apply(strip_short)
 
#strip links
data_rm_2['data'] = data_rm_2['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

## Stopwords

In [25]:
# Apply the remove_stopwords function to the 'text' column using the apply method
#data_rm_2['data'] = data_rm_2['data'].apply(remove_stopwords_from_text)

data_rm_2['data'] = data_rm_2['data'].apply(lambda x: remove_stopwords(x, german_stop_words))

In [26]:
data_rm_2['data'].iloc[0]

'trainer xabi alonso mitgereisten fans monaco glucksmoment exequiel palacios hebt tor euro highlight geht bundesliga sonntag uhr freiburg'

## Using wordgoups 
Matching first and last name to just last name

In [28]:
data_rm_2 = name_wordgroups(data_rm_2)


## Delete empty rows and store csv

In [29]:
data_rm_2.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_2.csv', index=False)